In [ ]:
import pandas as pd
from data_tranform_class import DataTransform
from db_utils import RDSDatabaseConnector 


rds_connector = RDSDatabaseConnector()

rds_connector.connect()

df = rds_connector.load_data_from_csv(file_path='loan_payments_whole.csv')


In [ ]:
data_transformer = DataTransform(df)

date_columns = ['issue_date', 'earliest_credit_line', 'last_payment_date', 'next_payment_date', 'last_credit_pull_date']
categorical_columns = ['employment_length', 'home_ownership', 'verification_status', 'loan_status',
                        'payment_plan', 'purpose', 'grade', 'sub_grade', 'application_type', 'term']

data_transformer.convert_dates_to_datetime(date_columns)
data_transformer.convert_categorical_columns(categorical_columns)


rds_connector.save_data_to_csv(data_transformer.df, file_path='updated_loan_payments.csv', index=False)


In [ ]:
from data_frame_info import DataFrameInfo


df_info = DataFrameInfo(df) 


# print(df_info.count_null_values()) # looking at which the percentages of null values for each column

# columns to impute: funded_amount (5.5%), term (8.7%) int_rate (9.5%), employment length (3.9%), 
# drop null rows: last_payment_date (0.13%), last_credit_pull_date (0.013%) collections_12_mths_ex_med (0.09%)
# columns to drop (high number of NULLS): mths_since_last_delinq (57%), mths_since_last_record (88%), next_payment_date (60%), mths_since_last_major_derog (86%)

In [ ]:

# dropping the null rows


columns_to_drop_null_rows = ['last_payment_date', 'last_credit_pull_date', 'collections_12_mths_ex_med']

data_transformer.drop_rows(dropped_rows=columns_to_drop_null_rows)

# print(data_transformer.df.info())


rds_connector.save_data_to_csv(data_transformer.df, file_path='updated_loan_payments.csv', index=False)



In [ ]:
# dropping columns with very high amount of nulls 57%+



columns_to_drop = ['mths_since_last_delinq', 'mths_since_last_record', 'next_payment_date', 'mths_since_last_major_derog']

data_transformer.drop_columns(columns_to_drop)

# print(data_transformer.df.info())


rds_connector.save_data_to_csv(data_transformer.df, file_path='updated_loan_payments.csv', index=False)


In [ ]:
from plotter_class import Plotter

# data_plotter = Plotter(df)


# data_plotter.discrete_probability_distribution('term') # use median

# data_plotter.discrete_probability_distribution('employment_length') # use median




In [ ]:
data_transformer.impute_mode('term')
data_transformer.impute_mode('employment_length')


rds_connector.save_data_to_csv(data_transformer.df, file_path='updated_loan_payments.csv', index=False)

In [ ]:


# data_plotter = Plotter(df)


# data_plotter.histogram('int_rate') # mean
# data_plotter.kde_plot('funded_amount')  # mean

data_transformer.impute_mean('int_rate')
data_transformer.impute_mean('funded_amount')

rds_connector.save_data_to_csv(data_transformer.df, file_path='updated_loan_payments.csv', index=False)



In [ ]:
df_updated = rds_connector.load_data_from_csv(file_path='updated_loan_payments.csv')

df_info = DataFrameInfo(df_updated) 

# print(df_updated.columns)


# print(df_info.count_null_values())




# numeric_data = ['loan_amount', 'funded_amount', 'funded_amount_inv', 'int_rate', 'instalment', 'annual_inc', 'dti', 
# 'delinq_2yrs', 'inq_last_6mths', 'open_accounts', 'total_accounts', 'out_prncp', 'out_prncp_inv', 'total_payment', 
# 'total_payment_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_payment_amount', 'collections_12_mths_ex_med']

In [ ]:
# visualising removal of NULL values


# data_plotter.compare_distributions('funded_amount', df, df_updated)


In [ ]:
# data_plotter_updated = Plotter(df_updated)

# numeric_features = ['loan_amount', 'funded_amount', 'funded_amount_inv', 'int_rate', 'instalment', 'annual_inc', 'open_accounts', 'total_accounts', 'total_payment', 
# 'total_payment_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_payment_amount']

# data_plotter_updated.kde_hist_multi(numeric_features)


In [ ]:
from data_frame_info import DataFrameInfo

# df_info = DataFrameInfo(df_updated)

# df_info.data_skew(numeric_features)

# df_info.extract_statistical_values(numeric_features)

# judging by the kde plots and the skew data all columns on numeric_features but int_rate seem quite positively skewed

In [ ]:
# comparing boxcox and log transformations

# df_info.compare_transformations('total_rec_int')

# loan_amound - box cox, funded_amount - box cox, funded_amount_inv - log, int_rate box cox,  instalment box cox, annual_inc box cox, 
# open_accounts box cox, total_accounts box cox, total_payment box cox, total_payment_inv log, total_rec_prncp log, total_rec_int box cox,
# total_rec_late_fee log, recoveries log, collection_recovery_fee log, last_payment_amount log

In [ ]:
# transforming skew box cox columns

data_transformer = DataTransform(df_updated.copy())

data_transformer.boxcox_transform('loan_amount')
data_transformer.boxcox_transform('funded_amount')
data_transformer.boxcox_transform('int_rate')
data_transformer.boxcox_transform('instalment')
data_transformer.boxcox_transform('annual_inc')
data_transformer.boxcox_transform('open_accounts')
data_transformer.boxcox_transform('total_accounts')
data_transformer.boxcox_transform('total_payment')
data_transformer.boxcox_transform('total_rec_int')

rds_connector.save_data_to_csv(data_transformer.df, file_path='updated_2_loan_payments.csv', index=False) # skew transformed csv file


In [ ]:
# transforming skew log columns

data_transformer.log_transform('funded_amount_inv')
data_transformer.log_transform('total_payment_inv')
data_transformer.log_transform('total_rec_prncp')
data_transformer.log_transform('total_rec_late_fee')
data_transformer.log_transform('recoveries')
data_transformer.log_transform('collection_recovery_fee')
data_transformer.log_transform('last_payment_amount')

rds_connector.save_data_to_csv(data_transformer.df, file_path='updated_2_loan_payments.csv', index=False) # skew transformed csv file


In [ ]:
# checking skew transformation

df_updated_2 = rds_connector.load_data_from_csv(file_path='updated_2_loan_payments.csv')

df_info = DataFrameInfo(df_updated_2)

numeric_features = ['loan_amount', 'funded_amount', 'funded_amount_inv', 'int_rate', 'instalment', 'annual_inc', 'open_accounts', 'total_accounts', 'total_payment', 
                    'total_payment_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_payment_amount']

# df_info.data_skew(numeric_features)

data_plotter = Plotter(df_updated_2)

# data_plotter.compare_distributions('last_payment_amount', df_updated, df_updated_2)

In [ ]:
# looking for outliers using box plots and hist

# data_plotter.box_plot('last_payment_amount')

# data_plotter.histogram('last_payment_amount')

# columns_to_remove_outliers = ['loan_amount', 'funded_amount', 'funded_amount_inv', 'int_rate', 'instalment', 'annual_inc', 'open_accounts', 'total_accounts', 'total_payment', 
# 'total_payment_inv', 'total_rec_prncp', 'total_rec_int']



In [273]:
# removing the outlies using the zscore method

data_transformer = DataTransform(df_updated_2.copy())

data_transformer.remove_outliers_zscore('loan_amount')
data_transformer.remove_outliers_zscore('funded_amount')
data_transformer.remove_outliers_zscore('funded_amount_inv')
data_transformer.remove_outliers_zscore('int_rate')
data_transformer.remove_outliers_zscore('instalment')
data_transformer.remove_outliers_zscore('annual_inc')
data_transformer.remove_outliers_zscore('open_accounts')
data_transformer.remove_outliers_zscore('total_accounts')
data_transformer.remove_outliers_zscore('total_payment')
data_transformer.remove_outliers_zscore('total_payment_inv')
data_transformer.remove_outliers_zscore('total_rec_prncp')
data_transformer.remove_outliers_zscore('total_rec_int')

rds_connector.save_data_to_csv(data_transformer.df, file_path='updated_3_loan_payments.csv', index=False) # no outliers csv file

AttributeError: 'DataTransform' object has no attribute 'remove_outliers_zscore'